In [1]:
from ggmodel_dev.models.water import EW
from ggmodel_dev.models.landuse import BE2, GE3, SL1_BE3

In [2]:
GE3.model_dictionnary

{'TMi_model': <ggmodel_dev.graphmodel.GraphModel at 0x7f646e50e610>,
 'TAi_model': <ggmodel_dev.graphmodel.GraphModel at 0x7f646e4edd50>,
 'M_xi_model': <ggmodel_dev.graphmodel.GraphModel at 0x7f646e4d1c50>,
 'TMP_CO2eq_model': <ggmodel_dev.graphmodel.GraphModel at 0x7f646e4d13d0>,
 'TMT_CO2eq_model': <ggmodel_dev.graphmodel.GraphModel at 0x7f646e498f50>,
 'TMA_CO2eq_model': <ggmodel_dev.graphmodel.GraphModel at 0x7f646e498e10>,
 'TEE_CO2eq_model': <ggmodel_dev.graphmodel.GraphModel at 0x7f646e498390>,
 'FE_CO2eq_model': <ggmodel_dev.graphmodel.GraphModel at 0x7f646e498fd0>,
 'GE3_partial_model': <ggmodel_dev.graphmodel.GraphModel at 0x7f646e4a6f90>,
 'GE3_model': <ggmodel_dev.graphmodel.GraphModel at 0x7f646e4a6cd0>}

In [3]:
GE3.model_properties

{'GE3_model': {'description': 'This model calculates the different agricultural non-CO2 emissions using emission factors. To compare the different greenhouse gasses, they are converted to their respective CO2-equivalent using their global-warming potential in relative to CO2. From the total animal populations that are calculated in the food demand (FDTi) model, methane (CH4) emissions from enteric fermentation are computed. The total animal stocks are also used to compute the total manure that is produced per animal. Total manure production is used to compute the CH4 and nitrous oxide (N2O) emissions from manure that is treated, left on the pasture, and applied to the soils. In addition, N2O emissions from fertilizer use and other CH4 and N2O emissions that are not yet included are added. After summing, all the emissions are divided by total population to compute the GE3 indicator:  the ratio of non-CO2 emissions in agriculture to population.',
  'display_name': 'Agricultural Non-CO2 E